In [1]:
from google.cloud import speech_v1p1beta1
from google.cloud.speech_v1p1beta1 import enums

import io

## not working
#from google.cloud import speech_v1
#from google.cloud.speech_v1 import enums

In [2]:
# Questa variabile deve contenere il path del file di credenziali
!echo "$GOOGLE_APPLICATION_CREDENTIALS"

/home/daniele/Documenti/Progetti/ASR/GoogleCloudSpeech/credentials.json


In [3]:
config_params = {
    "language_code": "it-IT", #"en-US"
    "sample_rate_hertz": 16000, #44100
    "encoding": enums.RecognitionConfig.AudioEncoding.MP3,
    #"encoding": enums.RecognitionConfig.AudioEncoding.LINEAR16,
}

In [4]:
def sample_recognize(audiofile_path, config, local=True):
    """
    Performs synchronous speech recognition on an audio file

    Arguments:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
      or
      local_file_path Path to local audio file, e.g. /path/audio.wav
    """

    client = speech_v1p1beta1.SpeechClient()

    if(local):
        with io.open(audiofile_path, "rb") as f:
            content = f.read()
        audio = {"content": content}
    else:
        # storage_uri = 'gs://cloud-samples-data/speech/brooklyn_bridge.mp3'
        storage_uri = audiofile_path
        audio = {"uri": storage_uri}
    
    response = client.recognize(config, audio)
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
        
    return alternative.transcript

In [11]:
filepath = 'parte1.mp3'
sample_recognize(filepath, config_params)

Transcript: aneddoti barzellette episodi


In [6]:
filepath = 'parte2.mp3'
sample_recognize(filepath, config_params)

Transcript: è una delle mie fobie quando prendo la macchina ho paura di non trovare parcheggio in centro


In [7]:
filepath = 'parte3.mp3'
sample_recognize(filepath, config_params)

Transcript: è una azione intenzionale Noi guardiamo di prof


In [12]:
filepath = 'Bri1.mp3'
sample_recognize(filepath, config_params)

Transcript: l'abbacchi e le galline perché so senza spine non so come baccalà


In [9]:
filepath = 'Bri2.mp3'
sample_recognize(filepath, config_params)

Transcript: spaghetti amatriciana


In [10]:
filepath = 'Bri2_2.mp3'
sample_recognize(filepath, config_params)

Transcript: Sì la socera more se famo du spaghetti amatriciana


# Test su mozilla dataset

In [5]:
import numpy as np

def _levenshtein_distance(ref, hyp):
    """Levenshtein distance is a string metric for measuring the difference
    between two sequences. Informally, the levenshtein disctance is defined as
    the minimum number of single-character edits (substitutions, insertions or
    deletions) required to change one word into the other. We can naturally
    extend the edits to word level when calculate levenshtein disctance for
    two sentences.
    """
    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]

def word_errors(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in word-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Levenshtein distance and word number of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)

def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    """Compute the levenshtein distance between reference sequence and
    hypothesis sequence in char-level.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Levenshtein distance and length of reference sentence.
    :rtype: list
    """
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)

def wer(reference, hypothesis, ignore_case=False, delimiter=' '):
    """Calculate word error rate (WER). WER compares reference text and
    hypothesis text in word-level. WER is defined as:
    .. math::
        WER = (Sw + Dw + Iw) / Nw
    where
    .. code-block:: text
        Sw is the number of words subsituted,
        Dw is the number of words deleted,
        Iw is the number of words inserted,
        Nw is the number of words in the reference
    We can use levenshtein distance to calculate WER. Please draw an attention
    that empty items will be removed when splitting sentences by delimiter.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param delimiter: Delimiter of input sentences.
    :type delimiter: char
    :return: Word error rate.
    :rtype: float
    :raises ValueError: If word number of reference is zero.
    """
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case,
                                         delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
    """Calculate charactor error rate (CER). CER compares reference text and
    hypothesis text in char-level. CER is defined as:
    .. math::
        CER = (Sc + Dc + Ic) / Nc
    where
    .. code-block:: text
        Sc is the number of characters substituted,
        Dc is the number of characters deleted,
        Ic is the number of characters inserted
        Nc is the number of characters in the reference
    We can use levenshtein distance to calculate CER. Chinese input should be
    encoded to unicode. Please draw an attention that the leading and tailing
    space characters will be truncated and multiple consecutive space
    characters in a sentence will be replaced by one space character.
    :param reference: The reference sentence.
    :type reference: basestring
    :param hypothesis: The hypothesis sentence.
    :type hypothesis: basestring
    :param ignore_case: Whether case-sensitive or not.
    :type ignore_case: bool
    :param remove_space: Whether remove internal space characters
    :type remove_space: bool
    :return: Character error rate.
    :rtype: float
    :raises ValueError: If the reference length is zero.
    """
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

In [6]:
from pathlib import Path
path_data = Path("../voice.mozilla.org_it_datasets/it")
test_dir = path_data / "test_mozillaData"
clips_dir = test_dir / 'clips'
labels_dir = test_dir / 'labels'

import sys
sys.path.append('../pytorch_ASR')

### Decode with google ASR

In [7]:
transcriptions = {}
audiofiles = clips_dir.glob('*.mp3')
for i,a in enumerate(audiofiles):
    t = sample_recognize(a, config_params)
    transcriptions[a.name.replace('.mp3','')] = t
    print(str(i) + "..." + t)

Transcript: la sua pittura accarezza il flusso del tempo
0...la sua pittura accarezza il flusso del tempo
Transcript: è una delle prime località della Lessinia che si incontrano Provenendo dalla città
1...è una delle prime località della Lessinia che si incontrano Provenendo dalla città
Transcript: in un tempo molto breve difese la sua dissertazione di laurea
2...in un tempo molto breve difese la sua dissertazione di laurea
Transcript: ma non riusciva a confidarsi con nessuno
3...ma non riusciva a confidarsi con nessuno
Transcript: con l'estate il ragazzo si trasferisce dalla nonna a Treviglio
4...con l'estate il ragazzo si trasferisce dalla nonna a Treviglio
Transcript: significato di immortale il nome È analogo a Kalid e Attanasio
5...significato di immortale il nome È analogo a Kalid e Attanasio
Transcript: nella trasmissione diventa subito uno dei beniamini del pubblico
6...nella trasmissione diventa subito uno dei beniamini del pubblico
Transcript: il primo corso per paracadutisti

ServiceUnavailable: 503 408:Request Timeout

In [26]:
import pickle
a_file = open("transcriptions_google_1000.pkl", "wb")
pickle.dump(transcriptions, a_file)
a_file.close()

#a_file = open("transcriptions_google.pkl", "rb")
#transcriptions = pickle. load(a_file)
#print(output)
#a_file. close()


### Load labels from Mozilla dataset and compare

In [30]:
labelfiles = labels_dir.glob('*.txt')
labels = {}
for i,l in enumerate(labelfiles):
    file = open(l, "r") 
    labels[l.name.replace('.txt','')] = file.read() 
    file.close() 

In [42]:
targets = []
preds = []
for k in transcriptions.keys():
    targets.append(labels[k])
    preds.append(transcriptions[k])

In [46]:
test_cer, test_wer = [], []
for k in range(len(targets)):
    test_cer.append(cer(preds[k], targets[k]))
    test_wer.append(wer(preds[k], targets[k]))
    
avg_cer = sum(test_cer)/len(test_cer)
avg_wer = sum(test_wer)/len(test_wer)
print(f'Character error rate: {avg_cer:.3f}')
print(f'Word error rate: {avg_wer:.3f}')

Character error rate: 0.093
Word error rate: 0.258
